In [171]:
import random
import numpy as np
from data_process import get_CIFAR10_data
from scipy.spatial import distance
%matplotlib inline
from save_submission import output_submission_csv

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [172]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 40000
VAL_IMAGES = 10000

In [173]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))

Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [174]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))

In [175]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/Perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


# Model Perceptron

In [193]:
import numpy as np


class Perceptron:
    def __init__(self, n_class: int, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            n_class: the number of classes
            lr: the learning rate
            epochs: the n
            umber of epochs to train for
        """
        self.w = 0
        self.lr = lr
        self.epochs = epochs
        self.n_class = n_class
        self.bias = 0
        self.activation = self._activation

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.
        Use the perceptron update rule as introduced in Lecture 3.
        Parameters:
            X_train: a number array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        self.w = np.random.normal(loc=0.0,scale=1.0,size=D)

        ###### YOUR CODE STARTS HERE ######
        self.errors = []
        for epoch in range(self.epochs):
            epoch_l = 0
            for x_i,target in zip(X_train,y_train):
                pred = self.predict(x_i)
                pred_err = target - pred
                update = self.lr * pred_err
                epoch_l += pred_err ** 2
                self.w += update * x_i
                self.bias += update
        ###### YOUR CODE ENDS HERE ######
        pass
    def _activation(self,x):
        return np.where(x>=0,1,0)
    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        return self.activation(np.dot(X_test,self.w.T))


## Train Perceptron on CIFAR

In [194]:
lr = 0.5
n_epochs = 10

percept_CIFAR = Perceptron(n_class_CIFAR, lr, n_epochs)
percept_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

In [195]:
pred_percept = percept_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_CIFAR)))

The training accuracy is given by: 12.150000


### Validate Perceptron on CIFAR

In [196]:
pred_percept = percept_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_CIFAR)))

The validation accuracy is given by: 12.150000


### Test Perceptron on CIFAR

In [197]:
pred_percept = percept_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_CIFAR)))

The testing accuracy is given by: 12.450000


In [198]:
output_submission_csv('output/Perceptron_submission_CIFAR.csv', percept_CIFAR.predict(X_test_CIFAR))

# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.



You will implement the Logistic Classifier in the **models/Logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [212]:
"""Logistic regression model."""

import numpy as np
import math


class Logistic:
    def __init__(self, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.w = 0 ###### TODO: change this
        self.lr = lr ###### TODO: change this
        self.epochs = epochs ###### TODO: change this
        self.threshold = 5 ###### TODO: change this
        self.bias = 0
    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        """Sigmoid function.
        Parameters:
            z: the input
        Returns:
            the sigmoid of the input
        """
        return 1 / ( 1+ np.exp(-z))

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.
        Use the logistic regression update rule as introduced in lecture.
        Parameters:
            X_train: a numpy array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        self.w = np.random.normal(loc=0.0,scale=1.0,size=D)

        ##### YOUR CODE STARTS HERE #####
        for _ in range(self.epochs):
            linear_model = np.dot(X_train,self.w) + self.bias
            y_predicted = self.sigmoid(linear_model)
            dW = (1/N)* np.dot(X_train.T,(y_predicted - y_train))
            db = (1/N) * np.sum(y_predicted - y_train)
            self.w -= self.lr * dW
            self.bias -= self.lr *db
        ##### YOUR CODE ENDS HERE #####
        
    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        y_test = np.zeros(N)
        ##### YOUR CODE STARTS HERE #####
        linear_model = np.dot(X_test,self.w) + self.bias
        y_predicted = self.sigmoid(linear_model)
        y_test = [1 if i > 0.5 else 0 for i in y_predicted]
        ##### YOUR CODE ENDS HERE #####
        return y_test

### Training Logistic Classifer

In [213]:
learning_rate = 0.01
n_epochs = 1000

lr = Logistic(learning_rate, n_epochs)
lr.train(X_train_CIFAR, y_train_CIFAR)

C:\Users\sk031\AppData\Local\Temp/ipykernel_15648/3864971174.py:26: RuntimeWarning: overflow encountered in exp
  return 1 / ( 1+ np.exp(-z))


In [214]:
pred_lr = lr.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_CIFAR)))

The training accuracy is given by: 12.127500


C:\Users\sk031\AppData\Local\Temp/ipykernel_15648/3864971174.py:26: RuntimeWarning: overflow encountered in exp
  return 1 / ( 1+ np.exp(-z))


### Validate Logistic Classifer

In [215]:
pred_lr = lr.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_CIFAR)))

The validation accuracy is given by: 12.160000


C:\Users\sk031\AppData\Local\Temp/ipykernel_15648/3864971174.py:26: RuntimeWarning: overflow encountered in exp
  return 1 / ( 1+ np.exp(-z))


### Test Logistic Classifier

In [216]:
pred_lr = lr.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_CIFAR)))

lr

output_submission_csv('output/Logistic_submission_CIFAR.csv', lr.predict(X_test_CIFAR))

The testing accuracy is given by: 12.410000


C:\Users\sk031\AppData\Local\Temp/ipykernel_15648/3864971174.py:26: RuntimeWarning: overflow encountered in exp
  return 1 / ( 1+ np.exp(-z))
